# MongoDB

## Терминология

**MongoDB** — документоориентированная система управления базами данных, не требующая описания схемы таблиц.

Считается одним из классических примеров NoSQL-систем. 

Использует JSON-подобные документы и схему базы данных.

Написана на языке C++. 

Система поддерживает ad-hoc-запросы: они могут возвращать конкретные поля документов и пользовательские JavaScript-функции. 

*Ad hoc — латинская фраза, означающая «специально для этого», «по особому случаю».

Поддерживается поиск по регулярным выражениям. Также можно настроить запрос на возвращение случайного набора результатов.

Имеется поддержка индексов (объект базы данных, создаваемый с целью повышения производительности поиска данных).

**База данных** — это физический контейнер для коллекций.

**Коллекция** — группа документов MongoDB. В терминологии SQL это соответствует таблице.

**Документ** — запись в коллекции MongoDB, набор пар ключ-значение. В терминологии SQL это похоже на строку в таблице базы данных.

**Поле** — ключ в документе. В терминологии SQL похоже на столбец в таблице.

**Встроенный документ** — в терминологии SQL похоже на создание связей между несколькими таблицами, по которым разбросаны данные, что делается операциями JOIN.

<center><img src="https://raw.githubusercontent.com/ryndovaira-org/data_science_notes/main/images/MongoDB_database_colection.png" width="600" height="600"/></center>

В июне 2018 года (в версии 4.0) добавлена поддержка транзакций, удовлетворяющих требованиям ACID.

### MongoDB подходит для следующих применений
- хранение и регистрация событий;
- системы управления документами и контентом;
- электронная коммерция;
- игры;
- данные мониторинга, датчиков;
- мобильные приложения;
- хранилище операционных данных веб-страниц (например, хранение комментариев, рейтингов, профилей пользователей, сеансы пользователей).

# References

## Подготовка окружения

### Установка MongoDB

[MongoDB Community Server](https://www.mongodb.com/try/download/community)

[Robo 3T: the hobbyist GUI](https://robomongo.org/download)

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

pip 20.3.3 from /home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!conda install pymongo -y

Solving environment: done

## Package Plan ##

  environment location: /home/ira/anaconda3/envs/LevelUp_DataScience

  added / updated specs:
    - pymongo


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.12.~ --> pkgs/main::ca-certificates-2021.1.19-h06a4308_1


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


# References

## MongoDB Типы данных

Документы в MongoDB являются **"JSON-like"**.

- `null`. Может использоваться для представления нулевого значения или несуществующего поля.
```
{ x: null }
```


- `boolean`. Это два значения true false.
```
{ x: true, y: false }
```


- `number`. Числа используются формат 64 битных с плавающей точкой. Для целых чисел можно использовать функции `NumberInt()` или `NumberLong()` которые используют 4 байта и 8 байтов для хранения чисел соответственно. `NumberDecimal()` принимает десятичное значение в виде строки.
```
{
 x: 3.14,
 y: 3,
 z: NumberInt('3'),
 r: NumberLong('2090845886852'),
 m: NumberDecimal("1000.55")
}
```


- `string`. Строки это массив UTF-8 символов.
```
{ x: 'string' }
```


- `date`. Дата в mongoDB это 64-битные целые числа, которые показывают дату прошедшую с эпохи Unix (англ. Unix Epoch) (1 января, 1970 года). Time zone не сохраняется. Для работы с датой mongoDB используют класс Date в JS. Дата в mongo shell используют текущую time zone, но в базе хранится значение миллисекунд прошедших с эпохи линукс.
```
{ 
x: new Date(),
y: new ISODate("2012-12-19T06:01:17.171Z")
}
```


- `regular expression`. Сохраненные регулярные выражения могут использоваться в регулярных выражениях в JS.
```
{ x: /foobar/i }
```


- `array`. Наборы (sets) или Списки (lists) могут быть представлены как массивы в базе. MongoDB поддерживает атомный update элементов. Например если в целом массиве нужно заменить элемент pie на pi, то мы заменим только этот элемент.
```
{ x: ['string', 3.14, new Date()] }
```


- `embedded document`. Документы могут содержать целые документы, встроенные в качестве значений в родительский документ. MongoDB так же как и с массивами понимает, когда используется embedded documents и оптимизирует работу с ними.
```
{ x: { name: 'Merrick', isAdmin: true } }
```


- `object id`. Идентификатор объекта это отдельный 12-байтовый тип данных, который используется для идентифицирования документов.
    - Каждый документ должен иметь поле _id, поле может быть любого типа, но обычно оно типа `ObjectId`. 
    - Тип `ObjectId` был разработан, легким (lightweight) в тоже время, что бы мог генерировать уникальные ключи на разных платформах. 
    - Если создать несколько `object id` в быстрой последовательности, можно заметить, что только последние несколько цифр меняются каждый раз. Это связано с тем как создаются `ObjectId`. 12 байтов формируются следующим образом:
        - Первые 4 байта это timestamp. В этой информации присутствует неявно дата создания документа.
        - Следующие 3 байта это machine PID, что означает уникальный идентификатор машины, сделано для того, чтобы разные машины случайно не создали одинаковый `ObjectId`. Обычно это хэш машины.
        - Чтобы обеспечить уникальность между процессами, следующие два байта берутся из PID процесса.
        - Остальные три байта это просто инкрементация, которая отвечает за уникальность выполнения в данную секунду, что позволяет нам создавать 16,777,216 `ObjectId` в секунду на одной машине в одном процессе.

```
{ x: ObjectId() }
{ _id: 10, calc: NumberLong("2090845886852") }
```


- `binary data`. Двоичные данные представляют собой строку произвольных байтов. Ими нельзя манипулировать из оболочки. Двоичные данные - единственный способ хранить строки не-UTF-8 в базе данных.


- `code`. MongoDB позволяет хранить произвольный JS код в документах и запросах под отдельным типом данных.
```
{ x: function list() { /* ... */ } }
```

# References

## MongoDB + Python

In [3]:
import pymongo
from pymongo import MongoClient

pymongo.__version__

'3.11.3'

# References

### Подключение к базе данных

In [4]:
# данные для подключения к БД
host = "localhost"
port = 3306

In [5]:
# альтернативный вариант: MongoClient(host, port)
client = MongoClient(f"mongodb://{host}:{port}/")
client

MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True)

In [6]:
db = client["test-database"]
db

Database(MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True), 'test-database')

In [7]:
# альтернативный вариант: collection = db['test_collection']
collection = db.test_collection
collection

Collection(Database(MongoClient(host=['localhost:3306'], document_class=dict, tz_aware=False, connect=True), 'test-database'), 'test_collection')

# References

### Создание, обновление и удаление документов. Запросы.

Данные в MongoDB представлены (и хранятся) с использованием документов в стиле JSON.

В PyMongo используются словари (dict) для представления документов.

- `insert_<one|many>` - базовый метод для добавление информации.
- `delete_<one|many>` - базовый метод для удаления информации.
- `update` - базовый метод для обновления информации.
- `find[_one]` - аналог SELECT в MySQL, используется для выборки документов.

#### Селекторы запросов

**Сравнение (Comparison)**
- `$eq` Соответствует значениям, которые равны указанному значению.
- `$gt` Соответствует значениям, превышающим указанное значение.
- `$gte` Соответствует значениям, которые больше или равны указанному значению.
- `$in` Соответствует любому из значений, указанных в массиве.
- `$lt` Соответствует значениям, которые меньше указанного значения.
- `$lte` Соответствует значениям, которые меньше или равны указанному значению.
- `$ne` Соответствует всем значениям, которые не равны указанному значению.
- `$nin` Не соответствует ни одному из значений, указанных в массиве.


**Логические (Logical)**
- `$and` Объединяет предложения запроса (clauses) с помощью логического И возвращает все документы, соответствующие условиям обоих предложений.
- `$not` Инвертирует эффект выражения запроса и возвращает документы, которые не соответствуют выражению запроса.
- `$nor` Объединяет предложения запроса с логическим ИЛИ-НЕ возвращает все документы, которые не соответствуют обоим предложениям.
- `$or` Объединяет предложения запроса с помощью логического ИЛИ, возвращает все документы, соответствующие условиям любого предложения.


**Для элементов (Element)**
- `$exists` Соответствует документам, имеющим указанное поле.
- `$type` Выбирает документы, если поле имеет указанный тип.


**Вычисляемые (Evaluation)**
- `$expr` Позволяет использовать выражения агрегирования в языке запросов.
- `$jsonSchema` Проверяет документы на соответствие данной схеме JSON.
- `$mod` Выполняет операцию по модулю над значением поля и выбирает документы с указанным результатом.
- `$regex` Выбирает документы, значения которых соответствуют указанному регулярному выражению.
- `$text` Выполняет текстовый поиск.
- `$where` Соответствует документам, удовлетворяющим выражению JavaScript.


**Для массивов (Array)**
- `$all` Соответствует массивам, содержащим все элементы, указанные в запросе.
- `$elemMatch` Выбирает документы, если элемент в поле массива соответствует всем указанным условиям `$elemMatch`.
- `$size` Выбирает документы, если поле массива имеет указанный размер.


**Битовые (Bitwise)**
- `$bitsAllClear` Соответствует числовым или двоичным значениям, в которых все битовые позиции имеют значение 0.
- `$bitsAllSet` Соответствует числовым или двоичным значениям, в которых все позиции битов имеют значение 1.
- `$bitsAnyClear` Соответствует числовым или двоичным значениям, в которых любой бит из набора битовых позиций имеет значение 0.
- `$bitsAnySet` Соответствует числовым или двоичным значениям, в которых любой бит из набора битовых позиций имеет значение 1.


**Геопространственный (Geospatial)**
- `$geoIntersects` Выбирает геометрии, которые пересекаются с геометрией `GeoJSON`. Индекс `2dsphere` поддерживает `$geoIntersects`.
- `$geoWithin` Выбирает геометрию в пределах геометрии `GeoJSON`. Индексы `2dsphere` и `2d` поддерживают `$geoWithin`.
- `$near` Возвращает геопространственные объекты в непосредственной близости от точки. Требуется геопространственный индекс. Индексы `2dsphere` и `2d` поддерживают `$near`.
- `$nearSphere` Возвращает геопространственные объекты в непосредственной близости от точки на сфере. Требуется геопространственный индекс. Индексы `2dsphere` и `2d` поддерживают `$nearSphere`.



#### Операторы обновления (Update Operators)

```
{
   <operator1>: { <field1>: <value1>, ... },
   <operator2>: { <field2>: <value2>, ... },
   ...
}
```

**Поля (Fields)**
- `$currentDate` Устанавливает значение поля на текущую дату в виде даты или метки времени.
- `$inc` Увеличивает значение поля на указанную величину.
- `$min` Обновляет поле, только если указанное значение меньше существующего значения поля.
- `$max` Обновляет поле только в том случае, если указанное значение больше существующего значения поля.
- `$mul` Умножает значение поля на указанную величину.
- `$rename` Переименовывает поле.
- `$set` Устанавливает значение поля в документе.
- `$setOnInsert` Устанавливает значение поля, если обновление приводит к вставке документа. Не влияет на операции обновления, которые изменяют существующие документы.
- `$unset` Удаляет указанное поле из документа.


**Массивы (Array)**
- `$` Действует как заполнитель для обновления первого элемента, соответствующего условию запроса.
- `$[]` Действует как заполнитель для обновления всех элементов в массиве для документов, соответствующих условию запроса.
- `$[<identifier>]` Действует как заполнитель для обновления всех элементов, которые соответствуют условию `arrayFilters` для документов, соответствующих условию запроса.
- `$addToSet` Добавляет элементы в массив, только если они еще не существуют в наборе.
- `$pop` Удаляет первый или последний элемент массива.
- `$pull` Удаляет все элементы массива, соответствующие указанному запросу.
- `$pullAll` Удаляет все совпадающие значения из массива.
- `$push` Добавляет элемент в массив.


**Модификаторы (Modifiers)**
- `$each` Изменяет операторы `$push` и `$addToSet` для добавления нескольких элементов для обновления массива.
- `$position` Изменяет оператор `$push`, чтобы указать позицию в массиве для добавления элементов.
- `$slice` Изменяет оператор `$push`, чтобы ограничить размер обновляемых массивов.
- `$sort` Изменяет оператор `$push`, чтобы отсортировать документы хранащиеся в массиве.


**Битовые (Bitwise)**
- `$bit` Выполняет побитовое обновление целочисленных значений AND, OR и XOR.

Продолжение в файлах [07_2_MongoDB.py](./07_2_MongoDB.py) и [07_3_MongoDB_with_scrapy_results.py](./07_3_MongoDB_with_scrapy_results.py)

# References

[Wikipedia MongoDB](https://ru.wikipedia.org/wiki/MongoDB)

[Большой туториал по MongoDB](https://medium.com/@Merrick_krg/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D1%82%D1%83%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB-%D0%BF%D0%BE-mongodb-7e2f9e17f0c7)

[MongoDB: Создание, обновление и удаление документов](https://habr.com/ru/post/134524/)

[MongoDB: Запросы](https://habr.com/ru/post/134590/)

[MongoDB vs MySQL: A Comparative Study on Databases](https://www.simform.com/mongodb-vs-mysql-databases)

[Academy 3T](https://studio3t.com/academy/)

[Robo 3T GUI](https://robomongo.org/download)

[PyMongo 3.11.2 documentation Tutorial](https://pymongo.readthedocs.io/en/stable/tutorial.html)

[Query and Projection Operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors)

[SQL to MongoDB Mapping Chart](https://docs.mongodb.com/manual/reference/sql-comparison/)

[MongoDB против MySQL](https://coderlessons.com/tutorials/bazy-dannykh/uchebnik-mongodb/20-mongodb-protiv-mysql#7)

[Comparing MongoDB vs MySQL](https://www.mongodb.com/compare/mongodb-mysql#:~:text=MySQL%20is%20a%20relational%20database,(SQL)%20for%20database%20access.&text=MongoDB%20is%20a%20NoSQL%20database,data%20as%20JSON%2Dlike%20documents.)

[MongoDB Vs MySQL: A Detailed Study To Choose The Best One?](https://www.thetechieshouse.com/mongodb-vs-mysql/)